In [ ]:
# Load Drive Helper
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Import các thư viên cần thiết
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.metrics import (precision_score, recall_score, f1_score, accuracy_score)

import tensorflow as tf

In [ ]:
DATASET_PATH = '/content/drive/My Drive/NT204/dataset/'
TRAINSET_PATH = DATASET_PATH + 'train_set.csv'
TESTSET_PATH = DATASET_PATH + 'test_set.csv'

# Tạo thư mục để tải dataset
if not os.path.exists(DATASET_PATH):
  os.makedirs(DATASET_PATH)

'''
Tải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.
Lưu ý: Đặt đúng tên các file.
- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing
'''


'\nTải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.\nLưu ý: Đặt đúng tên các file.\n- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing\n'

In [ ]:
# Tiền xử lý dữ liệu
lable_encoder = preprocessing.LabelEncoder()

# Tập train
data_train = pd.read_csv(TRAINSET_PATH)

x_train, y_train = data_train.iloc[:, :-1], data_train.iloc[:, [-1]]
y_train = y_train.apply(lable_encoder.fit_transform)
x_train, y_train = x_train.to_numpy(), y_train.to_numpy()

# Lấy số thuộc tính và số lớp
NUM_FEATURES = x_train.shape[1] # Số thuộc tính bằng số chiều (số cột của x_train hoặc x_test)
NUM_CLASSES = np.unique(y_train).size # Số class bằng tổng số các giá trị khác nhau của y_train

# one-hot encoding label
y_train = tf.keras.utils.to_categorical(y_train)

In [ ]:
print("Number of features: %s" %NUM_FEATURES)
print("Number of classes: %s" %NUM_CLASSES)

Number of features: 42
Number of classes: 11


In [ ]:
# Tạo mô hình deep learning
# Thêm 2 lớp Dense nữa và chỉnh sửa các tham số phù hợp
model = tf.keras.Sequential(
  [
    tf.keras.layers.Dense(1024, activation="relu", input_shape=(NUM_FEATURES,)),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile mô hình
model.compile(optimizer='adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
# Huấn luyện mô hình
# Giảm batch_size xuống 512 và tăng epochs lên 30
model.fit(x=x_train, y=y_train, batch_size=512, epochs=30, shuffle=True, validation_split=0.1)

Epoch 1/30
3126/3126 [==============================] - 14s 4ms/step - loss: 0.0477 - accuracy: 0.9805 - val_loss: 0.0521 - val_accuracy: 0.9796
Epoch 2/30
3126/3126 [==============================] - 14s 4ms/step - loss: 0.0475 - accuracy: 0.9805 - val_loss: 0.0520 - val_accuracy: 0.9797
Epoch 3/30
3126/3126 [==============================] - 13s 4ms/step - loss: 0.0474 - accuracy: 0.9806 - val_loss: 0.0523 - val_accuracy: 0.9793
Epoch 4/30
3126/3126 [==============================] - 14s 4ms/step - loss: 0.0471 - accuracy: 0.9807 - val_loss: 0.0515 - val_accuracy: 0.9799
Epoch 5/30
3126/3126 [==============================] - 13s 4ms/step - loss: 0.0469 - accuracy: 0.9807 - val_loss: 0.0524 - val_accuracy: 0.9797
Epoch 6/30
3126/3126 [==============================] - 14s 4ms/step - loss: 0.0470 - accuracy: 0.9807 - val_loss: 0.0521 - val_accuracy: 0.9797
Epoch 7/30
3126/3126 [==============================] - 16s 5ms/step - loss: 0.0467 - accuracy: 0.9807 - val_loss: 0.0519 - val_ac

In [ ]:
# Lưu mô hình
model.save('/content/drive/My Drive/NT204/model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load dữ liệu để kiểm tra mô hình
data_test = pd.read_csv(TESTSET_PATH)
x_test, y_test = data_test.iloc[:, :-1], data_test.iloc[:, [-1]]
y_test = y_test.apply(lable_encoder.fit_transform)
x_test, y_test = x_test.to_numpy(), y_test.to_numpy()

In [ ]:
# Kiểm tra mô hình với tập test
preds = model.predict(x_test, batch_size=1024)

745/745 [==============================] - 1s 2ms/step


In [ ]:
# Tính toán kết quả
y_pred = np.argmax(preds, axis=1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print('Accuracy: %s\nPrecision: %s\nRecal: %s\nF1-Score: %s' %(accuracy, precision, recall, f1))

Accuracy: 0.9798192948957698
Precision: 0.978955110177891
Recal: 0.9798192948957698
F1-Score: 0.9780732866826606
